# Projeto 2 - Buscas de Árvore Binária e AVL

- Autor: Victor Venites
- Data: 25/10/2021
- Disciplina: Algoritmos e Estrutura de Dados
- Curso: Mestrado Profissional em Computação Aplicada
- Instituição: Universidade Mackenzie

## Introdução

$\;\;\;\;$ O desafio neste projéto é fazer análises de dados, porem sem utilizar as bibliotecas conhecidas(Pandas ou até o Excel). Neste caso usamos indexação dos dados por meio das tecnicas de Busca de Árvore Binária(BST em inglês) e BST auto-balanceadas chamadas de AVL (dos inventores Adelson-Velsky and Landis). 

- Os testes foram feitos em um Notebook:
    - Windows 10 Pro
    - 16GB de memória RAM
    - Processador Core I5 7300HQ de sétima geração com 2.5GHz
    - Python 3.7.10

- Para as Árvores BST e AVL compilei os códigos do autor do artigo abaixo, onde ele mostra um benchmark de comparação:
    - https://medium.com/@aksh0001/avl-trees-in-python-bc3d0aeb9150
    - Autor: Akshay Kumar
    - Os comandos necesários para este experimento são chamados através das classe BST() e AVLTree()
    - Como diferencial em relação á outros artigos do assunto, aqui o autor cria atributor com chave e valor.
    - Ou seja, cada nó é um indice do dado da tabela guardado na variével key, e na variável val guardamos os indices das linhas em que este dado ocorre

---

## 1. Baixar CSV com dados

$\;\;\;\;$ Para base de dados será utilizada a "Breast Cancer de Wisconsin", que mostra um dataset de características para cancer "Maligno" = 4 e "Benigno" = 2. As demais colunas são dados exames sobre características de uma célula possivelmente cancerigena.

$\;\;\;\;$ Na leitura das linhas serão ignoradas as linhas com dados '?' no lugar de números

- https://www.kaggle.com/saptarsi/breastcancer
- https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)
- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [1]:
import csv

In [2]:
arquivo = "breast-cancer-wisconsin.data"
colunas = ["ID", "Clump_Thickness", "Cell_Size", "Cell_Shape", "Marginal_Adhesion", "Single_Epithelial_Cell_Size",
           "Bare_Nuclei", "Bland_Cromatin", "Normal_Nucleoli",
           "Mitoses", "Class"]

In [3]:
linhas = []
with open(arquivo, 'r') as arquivo_csv:
    csv_lido = csv.reader(arquivo_csv)
    
    # retira as linhas com dados ruins
    lista_aux = []
    for linha in csv_lido:
        if '?' not in linha:
            lista_aux.append(linha)
    print(f"Total de linhas {csv_lido.line_num} no CSV")
    
    for indice, linha in enumerate(lista_aux):
        linhas.append([indice, linha])
    
    print(f"Total de linhas {len(linhas)} na lista sem '?'")

Total de linhas 699 no CSV
Total de linhas 683 na lista sem '?'


In [4]:
linhas[:5]

[[0, ['1000025', '5', '1', '1', '1', '2', '1', '3', '1', '1', '2']],
 [1, ['1002945', '5', '4', '4', '5', '7', '10', '3', '2', '1', '2']],
 [2, ['1015425', '3', '1', '1', '1', '2', '2', '3', '1', '1', '2']],
 [3, ['1016277', '6', '8', '8', '1', '3', '4', '3', '7', '1', '2']],
 [4, ['1017023', '4', '1', '1', '3', '2', '1', '3', '1', '1', '2']]]

---

## 2. Escrever rotinas para ler o arquivo e mapear os dados em uma Árvore Binária de Busca E AVL (com as características da base de dados). Vocês devem pensar na melhor forma de montar a estrutura de dados e o que carregar na estrutura.

- Iterar pela linha, e iterar pela coluna inserindo na arvore de cada coluna em um dicionário chave/valor do python

In [5]:
from bst_e_avl import *

In [6]:
def mapear_df(dados, arvore_index):
    dicionario_colunas = dict()

    for coluna in colunas:
        dicionario_colunas.update({coluna: arvore_index()})
        
    for indice, linha in linhas:
        #print(f"linha {indice}")
        for coluna, dado in zip(colunas, linha):
            #Chave, valor
            dicionario_colunas[coluna].insert(int(dado), [indice])
            if dicionario_colunas[coluna].get(int(dado)).val != [indice]:
                dicionario_colunas[coluna].get(int(dado)).val.append(indice)

    return dicionario_colunas

In [7]:
import time
import pandas as pd

In [8]:
df_benchmark = pd.DataFrame(columns = ['Teste', 'Colunas', 'Tempo BST', 'Tempo AVL'])
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL


In [9]:
Tempo_inicial_insert_BST = time.time()
indexacao_bst = mapear_df(linhas, BST)
Tempo_final_insert_BST = time.time() - Tempo_inicial_insert_BST
Tempo_final_insert_BST

0.09299969673156738

In [10]:
Tempo_inicial_insert_AVL = time.time()
indexacao_avl = mapear_df(linhas, AVLTree)
Tempo_final_insert_AVL = time.time() - Tempo_inicial_insert_AVL
Tempo_final_insert_AVL

0.05699777603149414

In [11]:
salvar = ["Indexação do CSV", "Todas", Tempo_final_insert_BST, Tempo_final_insert_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093,0.056998


---

## 3. Escrever métodos para análise de dados, como buscar determinados atributos(data science). Cada grupo irá planejar as questões que deseja responder sobre os dados mapeados na estrutura montada.

$\;\;\;\;$ Para esta etapa serão usados como inspiração os comandos conhecidos do Pandas e do T-SQL.

- Saber a altura das indexações das colunas
- Filtrar com condição de um dado especifico de uma coluna, e depois de todas
- Re-ordenar a base de dados de acordo com a ordem de uma coluna, e depois de todas
- Contar a frequencia dos dados na coluna, e depois de todas
- Listar os dados estatisticos simples, como máximo, minimo, média, etc...

### 3.1. Trazer a altura das arvores de todas as colunas

In [12]:
indexacao_bst["ID"].get_height()

265

In [13]:
indexacao_avl["ID"].get_height()

11

In [14]:
def altura_arvore(lista_indexada, coluna):
    resultado = lista_indexada[coluna].get_height()
    return resultado

In [15]:
Tempo_inicial_altura_BST = time.time()
altura_arvore(indexacao_bst, "ID")
Tempo_final_altura_BST = time.time() - Tempo_inicial_altura_BST

In [16]:
Tempo_inicial_altura_AVL = time.time()
altura_arvore(indexacao_avl, "ID")
Tempo_final_altura_AVL = time.time() - Tempo_inicial_altura_AVL

In [17]:
salvar = ["Altura da Arvore", "ID", Tempo_final_altura_BST, Tempo_final_altura_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093,0.056998
1,Altura da Arvore,ID,0.000,0.000000


In [18]:
Tempo_inicial_altura_BST = time.time()
for coluna in list(indexacao_bst.keys()):
    altura_arvore(indexacao_bst, coluna)
Tempo_final_altura_BST = time.time() - Tempo_inicial_altura_BST

In [19]:
Tempo_inicial_altura_AVL = time.time()
for coluna in list(indexacao_avl.keys()):
    altura_arvore(indexacao_avl, coluna)
Tempo_final_altura_AVL = time.time() - Tempo_inicial_altura_AVL

In [20]:
salvar = ["Altura da Arvore", "Todas", Tempo_final_altura_BST, Tempo_final_altura_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998


### 3.2. Filtro com condição:
    - Select * WHERE indexacao_avl["Mitoses"] == 10
    - dicionario_colunas[coluna].get(dado).val

In [21]:
indices_filtro = indexacao_avl["Mitoses"].get(10).val
indices_filtro

[63, 69, 83, 96, 161, 181, 229, 231, 277, 290, 346, 467, 597, 632]

In [22]:
# Exemplo de Dado
linhas[-1]

[682, ['897471', '4', '8', '8', '5', '4', '5', '10', '4', '1', '4']]

In [23]:
def filtrar_base(lista, lista_indexada, coluna, valor):
    indices_filtro = lista_indexada[coluna].get(valor).val
    resultados = [["Indice", colunas]]
    resultados.extend([lista[x] for x in indices_filtro])
    return resultados

In [24]:
filtrar_base(linhas, indexacao_bst, "Mitoses", 10)

[['Indice',
  ['ID',
   'Clump_Thickness',
   'Cell_Size',
   'Cell_Shape',
   'Marginal_Adhesion',
   'Single_Epithelial_Cell_Size',
   'Bare_Nuclei',
   'Bland_Cromatin',
   'Normal_Nucleoli',
   'Mitoses',
   'Class']],
 [63, ['1116998', '10', '4', '2', '1', '3', '2', '4', '3', '10', '4']],
 [69, ['1123061', '6', '10', '2', '8', '10', '2', '7', '8', '10', '4']],
 [83, ['1147748', '5', '10', '6', '1', '10', '4', '4', '10', '10', '4']],
 [96, ['1165926', '9', '6', '9', '2', '10', '6', '2', '9', '10', '4']],
 [161, ['1198128', '10', '8', '10', '10', '6', '1', '3', '1', '10', '4']],
 [181, ['1206841', '10', '5', '6', '10', '6', '10', '7', '7', '10', '4']],
 [229, ['1241559', '10', '8', '8', '2', '8', '10', '4', '8', '10', '4']],
 [231, ['1242364', '8', '10', '10', '8', '6', '9', '3', '10', '10', '4']],
 [277, ['529329', '10', '10', '10', '10', '10', '10', '4', '10', '10', '4']],
 [290, ['640744', '10', '10', '10', '7', '9', '10', '7', '10', '10', '4']],
 [346, ['877291', '6', '10', '10'

In [25]:
Tempo_inicial_filtro_BST = time.time()
filtrar_base(linhas, indexacao_bst, "Mitoses", 10)
Tempo_final_filtro_BST = time.time() - Tempo_inicial_filtro_BST

In [26]:
Tempo_inicial_filtro_AVL = time.time()
filtrar_base(linhas, indexacao_avl, "Mitoses", 10)
Tempo_final_filtro_AVL = time.time() - Tempo_inicial_filtro_AVL

In [27]:
salvar = ["Filtro == 10", "Mitoses", Tempo_final_filtro_BST, Tempo_final_filtro_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000


In [28]:
Tempo_inicial_filtro_BST = time.time()
for coluna in list(indexacao_bst.keys()):
    filtrar_base(linhas, indexacao_bst, coluna, indexacao_bst[coluna].get_max())
Tempo_final_filtro_BST = time.time() - Tempo_inicial_filtro_BST

In [29]:
Tempo_inicial_filtro_AVL = time.time()
for coluna in list(indexacao_avl.keys()):
    filtrar_base(linhas, indexacao_avl, coluna, indexacao_avl[coluna].get_max())
Tempo_final_filtro_AVL = time.time() - Tempo_inicial_filtro_AVL

In [30]:
salvar = ["Filtro == Maior", "Todas", Tempo_final_filtro_BST, Tempo_final_filtro_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001


### 3.3. Order By Dados da Coluna
    - print_in_order(indexacao_avl[coluna_exemplo].root)
    - SELECT * WHERE dado_coluna == [lista ordenada]

In [31]:
def print_in_order(root: TreeNode, lista_ordenada):
    """
    Prints a in-order traversal of a binary tree: Left, Root, Right
    :param root: root of tree
    :return: none
    :Time: O(N)
    :Space: O(N) stack space due to recursion
    """
    if root is None:
        return 

    print_in_order(root.left, lista_ordenada)
    #print(root.key)
    lista_ordenada.append(root.key)
    print_in_order(root.right, lista_ordenada)
    
    resultado = list(lista_ordenada)
    return resultado

def listar_dados_ordenados(lista_indexada):
    return print_in_order(lista_indexada, [])

In [32]:
# Exemplo: teste
listar_dados_ordenados(indexacao_avl["Mitoses"].root)

[1, 2, 3, 4, 5, 6, 7, 8, 10]

In [33]:
def order_by(linhas, lista_indexada, coluna):
    dados_unicos = listar_dados_ordenados(lista_indexada[coluna].root)
    lista_ordernada_by = []
    for dado in dados_unicos:
        lista_ordernada_by.extend(filtrar_base(linhas, lista_indexada, coluna, dado))
    return lista_ordernada_by

In [34]:
order_by(linhas, indexacao_bst, "Clump_Thickness")[:5]

[['Indice',
  ['ID',
   'Clump_Thickness',
   'Cell_Size',
   'Cell_Shape',
   'Marginal_Adhesion',
   'Single_Epithelial_Cell_Size',
   'Bare_Nuclei',
   'Bland_Cromatin',
   'Normal_Nucleoli',
   'Mitoses',
   'Class']],
 [6, ['1018099', '1', '1', '1', '1', '2', '10', '3', '1', '1', '2']],
 [10, ['1035283', '1', '1', '1', '1', '1', '1', '3', '1', '1', '2']],
 [13, ['1043999', '1', '1', '1', '1', '2', '3', '3', '1', '1', '2']],
 [23, ['1059552', '1', '1', '1', '1', '2', '1', '3', '1', '1', '2']]]

In [35]:
order_by(linhas, indexacao_avl, "Clump_Thickness")[:5]

[['Indice',
  ['ID',
   'Clump_Thickness',
   'Cell_Size',
   'Cell_Shape',
   'Marginal_Adhesion',
   'Single_Epithelial_Cell_Size',
   'Bare_Nuclei',
   'Bland_Cromatin',
   'Normal_Nucleoli',
   'Mitoses',
   'Class']],
 [6, ['1018099', '1', '1', '1', '1', '2', '10', '3', '1', '1', '2']],
 [10, ['1035283', '1', '1', '1', '1', '1', '1', '3', '1', '1', '2']],
 [13, ['1043999', '1', '1', '1', '1', '2', '3', '3', '1', '1', '2']],
 [23, ['1059552', '1', '1', '1', '1', '2', '1', '3', '1', '1', '2']]]

In [36]:
Tempo_inicial_ordenar_BST = time.time()
order_by(linhas, indexacao_bst, "Clump_Thickness")
Tempo_final_ordenar_BST = time.time() - Tempo_inicial_ordenar_BST

In [37]:
Tempo_inicial_ordenar_AVL = time.time()
order_by(linhas, indexacao_avl, "Clump_Thickness")
Tempo_final_ordenar_AVL = time.time() - Tempo_inicial_ordenar_AVL

In [38]:
salvar = ["Ordenar", "Clump_Thickness", Tempo_final_ordenar_BST, Tempo_final_ordenar_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001
5,Ordenar,Clump_Thickness,0.000000,0.000000


In [39]:
Tempo_inicial_ordenar_BST = time.time()
for coluna in list(indexacao_bst.keys()):
    order_by(linhas, indexacao_bst, coluna)
Tempo_final_ordenar_BST = time.time() - Tempo_inicial_ordenar_BST

In [40]:
Tempo_inicial_ordenar_AVL = time.time()
for coluna in list(indexacao_avl.keys()):
    order_by(linhas, indexacao_avl, coluna)
Tempo_final_ordenar_AVL = time.time() - Tempo_inicial_ordenar_AVL

In [41]:
salvar = ["Ordenar", "Todas", Tempo_final_ordenar_BST, Tempo_final_ordenar_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001
5,Ordenar,Clump_Thickness,0.000000,0.000000
6,Ordenar,Todas,0.029024,0.006999


### 3.4. Value Counts
    - len(dicionario_colunas[coluna].get(dado).val)

In [42]:
def value_counts(lista_indexada, coluna):
    dados_unicos = listar_dados_ordenados(lista_indexada[coluna].root)
    
    lista_valores_contados = [["Valor", "Frequencia"]]
    for dado in dados_unicos:
        lista_valores_contados.append([dado, len(lista_indexada[coluna].get(dado).val)])
    
    return lista_valores_contados

In [43]:
value_counts(indexacao_bst, "Clump_Thickness")

[['Valor', 'Frequencia'],
 [1, 139],
 [2, 50],
 [3, 104],
 [4, 79],
 [5, 128],
 [6, 33],
 [7, 23],
 [8, 44],
 [9, 14],
 [10, 69]]

In [44]:
value_counts(indexacao_avl, "Clump_Thickness")

[['Valor', 'Frequencia'],
 [1, 139],
 [2, 50],
 [3, 104],
 [4, 79],
 [5, 128],
 [6, 33],
 [7, 23],
 [8, 44],
 [9, 14],
 [10, 69]]

In [45]:
Tempo_inicial_ValueCount_BST = time.time()
value_counts(indexacao_bst, "Clump_Thickness")
Tempo_final_ValueCount_BST = time.time() - Tempo_inicial_ValueCount_BST

In [46]:
Tempo_inicial_ValueCount_AVL = time.time()
value_counts(indexacao_avl, "Clump_Thickness")
Tempo_final_ValueCount_AVL = time.time() - Tempo_inicial_ValueCount_AVL

In [47]:
salvar = ["ValueCount", "Clump_Thickness", Tempo_final_ValueCount_BST, Tempo_final_ValueCount_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001
5,Ordenar,Clump_Thickness,0.000000,0.000000
6,Ordenar,Todas,0.029024,0.006999
7,ValueCount,Clump_Thickness,0.000000,0.000000


In [48]:
Tempo_inicial_ValueCount_BST = time.time()
for coluna in list(indexacao_bst.keys()):
    value_counts(indexacao_bst, coluna)
Tempo_final_ValueCount_BST = time.time() - Tempo_inicial_ValueCount_BST

In [49]:
Tempo_inicial_ValueCount_AVL = time.time()
for coluna in list(indexacao_avl.keys()):
    value_counts(indexacao_avl, coluna)
Tempo_final_ValueCount_AVL = time.time() - Tempo_inicial_ValueCount_AVL

In [50]:
salvar = ["ValueCount", "Todas", Tempo_final_ValueCount_BST, Tempo_final_ValueCount_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001
5,Ordenar,Clump_Thickness,0.000000,0.000000
6,Ordenar,Todas,0.029024,0.006999
7,ValueCount,Clump_Thickness,0.000000,0.000000
8,ValueCount,Todas,0.030512,0.004001


### 3.5. Descrições de Dados Estatísticos:
    - Minimo
        - indexacao_avl[coluna].get_min()
    - Máximo
        - indexacao_avl[coluna].get_max()
    - Média
    - Moda

In [51]:
def media(lista_indexada, coluna):
    dados_unicos = listar_dados_ordenados(lista_indexada[coluna].root)
    quantidade_total = 0
    soma = 0
    for dado in dados_unicos:
        quantidade = len(lista_indexada[coluna].get(dado).val)
        quantidade_total = quantidade_total + quantidade
        soma = soma + dado * quantidade
    return soma / quantidade_total

In [52]:
def moda_coluna(lista_indexada, coluna):
    moda = ["0", 0]
    for valor, frequencia in value_counts(lista_indexada, coluna)[1:]:
        if frequencia > moda[1]:
            moda = [valor, frequencia]
    return moda[0]

In [53]:
def descricao_base(lista_indexada):
    descricao_colunas = [["Coluna", "Minimo", "Maximo", "Media", "Moda"]]
    for coluna, dados in lista_indexada.items():
        descricao_colunas.append([coluna,
                                  lista_indexada[coluna].get_min(),
                                  lista_indexada[coluna].get_max(),
                                  media(lista_indexada, coluna),
                                  moda_coluna(lista_indexada, coluna)])
    return descricao_colunas

In [54]:
df_bst = pd.DataFrame(descricao_base(indexacao_bst)).T
df_bst.index = df_bst[0]
df_bst = df_bst.iloc[:, 1:]
df_bst.columns = df_bst.iloc[0, :]
df_bst = df_bst.iloc[1:, :]
df_bst

Coluna,ID,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Cromatin,Normal_Nucleoli,Mitoses,Class
0,,,,,,,,,,,
Minimo,63375,1,1,1,1,1,1,1,1,1,2
Maximo,13454352,10,10,10,10,10,10,10,10,10,4
Media,1076720.22694,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
Moda,1182404,1,1,1,1,2,1,3,1,1,2


In [55]:
df_avl = pd.DataFrame(descricao_base(indexacao_avl)).T
df_avl.index = df_avl[0]
df_avl = df_avl.iloc[:, 1:]
df_avl.columns = df_avl.iloc[0, :]
df_avl = df_avl.iloc[1:, :]
df_avl

Coluna,ID,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Cromatin,Normal_Nucleoli,Mitoses,Class
0,,,,,,,,,,,
Minimo,63375,1,1,1,1,1,1,1,1,1,2
Maximo,13454352,10,10,10,10,10,10,10,10,10,4
Media,1076720.22694,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
Moda,1182404,1,1,1,1,2,1,3,1,1,2


In [56]:
Tempo_inicial_Descricao_BST = time.time()
descricao_base(indexacao_bst)
Tempo_final_Descricao_BST = time.time() - Tempo_inicial_Descricao_BST
Tempo_final_Descricao_BST

0.0579984188079834

In [57]:
Tempo_inicial_Descricao_AVL = time.time()
descricao_base(indexacao_avl)
Tempo_final_Descricao_AVL = time.time() - Tempo_inicial_Descricao_AVL
Tempo_final_Descricao_AVL

0.007999181747436523

In [58]:
salvar = ["Descricao Estatistica", "Todas", Tempo_final_Descricao_BST, Tempo_final_Descricao_AVL]
df_benchmark.loc[len(df_benchmark)] = salvar

---

## 4. Criar gráficos, tabelas ou outros recursos para demonstrar resultados sobre as questões verificadas, bem como uma análise textual sobre os resultados.

In [59]:
df_benchmark

,Teste,Colunas,Tempo BST,Tempo AVL
0,Indexação do CSV,Todas,0.093000,0.056998
1,Altura da Arvore,ID,0.000000,0.000000
2,Altura da Arvore,Todas,0.000999,0.000998
3,Filtro == 10,Mitoses,0.000000,0.000000
4,Filtro == Maior,Todas,0.001001,0.001001
5,Ordenar,Clump_Thickness,0.000000,0.000000
6,Ordenar,Todas,0.029024,0.006999
7,ValueCount,Clump_Thickness,0.000000,0.000000
8,ValueCount,Todas,0.030512,0.004001
9,Descricao Estatistica,Todas,0.057998,0.007999


$\;\;\;\;$ As duas técnicas se mostraram rápidas para a exeução das tarefas em uma base de dados de 699 linhas, com vantagem para a Árvore AVL que ganhou em todas rodadas. Algumas das rotinas forma tão rápidas que registraram 0s, então para poder obter mais informação, adicionei um loop que faz a rotina rodar em todas as colunas. Mesmo assim no caso do filtro o tempo registrado foi 0s.

$\;\;\;\;$ A Indexação foi a tarefa que mais demorou, sendo essa a inserção dos dados do CSV nas árvores. Em comparação pode-se ver que torna a busca mais rápida. Porem, novamente a arvore AVL se mostra com uma busca muito mais rápida.

In [60]:
df_benchmark.describe()

,Tempo BST,Tempo AVL
count,10.000000,10.000000
mean,0.021253,0.007800
std,0.032096,0.017548
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.001000,0.001000
75%,0.030140,0.006250
max,0.093000,0.056998


$\;\;\;\;$ Qualquer operação aqui se mostrou mais rápida do que 1 décimo de segundo. Mas pensando em um cenário de BigData com a Árvore AVL se faz necessária. Pois ao tirar a média dos tempos das rodadas BST e AVL, a BST se mostra quase 3 vezes mais lenta, além de variar mais nos tempos visto que o desvio padrão também é maior. O que demonstra mais estabílidade para a AVL.